In [ ]:
import sys
sys.path.insert(0, '../')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from ydata_profiling import ProfileReport
import io
import tabulate
import helpers_eda_inicial
import helpers_cbc

In [ ]:
cbc, df_carreras = helpers_cbc.get_data('../../../assets/bronze/CBC/Sitacad_Tesis (1).xlsx')
actas = pd.read_csv('../../../assets/bronze/FCEN/FCEN_oficial_v2/reportes_actas.csv')
personas = pd.read_csv('../../../assets/bronze/FCEN/FCEN_oficial_v3/reporte_personas.csv')

In [ ]:
helpers_eda_inicial.dataset_profiling(cbc, 'CBC_v2')
helpers_eda_inicial.dataset_profiling(actas, 'Actas_v2')
helpers_eda_inicial.dataset_profiling(personas, 'Personas_v3')

In [ ]:
helpers_eda_inicial.initial_eda(cbc, 'CBC')

In [ ]:
helpers_eda_inicial.initial_eda(actas, 'Actas')

In [ ]:
helpers_eda_inicial.initial_eda(personas, 'Personas')

# Se conectan bien?

In [ ]:
cbc['Dni'].nunique()

In [ ]:
actas['dni'].nunique()

In [ ]:
personas['dni'].nunique()

convierto los dni en numero

In [ ]:
cbc['Dni'] = cbc['Dni'].astype(str)

In [ ]:
np.isin(cbc['Dni'].unique(), actas['dni'].unique()).sum()

In [ ]:
np.isin(cbc['Dni'].unique(), personas['dni'].unique()).sum()

In [ ]:
np.isin(actas['dni'].unique(), personas['dni'].unique()).sum()

Todas las actas se conectan con alguna persona (es raro que el dni no lo pude pasar a int)

In [ ]:
personas['dni'] = personas['dni'].astype(str)
# Largo de cada string
personas['longitud'] = personas['dni'].str.len()
# Estadísticas descriptivas
print(personas['longitud'].describe())

In [ ]:
%matplotlib inline
# Graficar la distribución de longitudes
personas['longitud'].value_counts().sort_index().plot(kind='bar')

# Agregar títulos si querés
plt.title("Distribución de cantidad de caracteres por celda")
plt.xlabel("Cantidad de caracteres")
plt.ylabel("Frecuencia")
plt.grid(True)
plt.show()

In [ ]:
personas[personas['longitud'] !=8]['dni'].unique()

In [ ]:
personas.drop(columns=['longitud'], inplace=True)

# Nulos en la columna Nota

Si en resultado hay ‘Ausente’ es el único caso donde Nota tiene el valor nulo

In [ ]:
actas['nota'].isna().sum()

In [ ]:
actas[actas['nota'].isna()]['resultado'].unique()

In [ ]:
actas[actas['resultado'] == 'Ausente']['nota'].unique()

In [ ]:
actas[actas['resultado'] == 'Ausente']['nota'].isna().sum()

Vemos los valores posibles

In [ ]:
actas['nota'].unique()

vemos los datos según el tipo de acta

In [ ]:
actas[actas['tipo_acta'] == 'Acta de Examen']['nota'].value_counts(dropna=False)

Me llama la atención los casos donde no hay nota numerica, los miro

In [ ]:
actas[(actas['tipo_acta'] == 'Acta de Examen') & (actas['nota'] == 'Aprob.')]

Hay muchos de Exactas Programa, la nota de examen final no era numerica

In [ ]:
actas[(actas['materia'] == 'Exactas Programa')]['nota'].value_counts(dropna=False)

In [ ]:
actas[(actas['materia'] == 'Exactas Programa') & (actas['nota'] == 'Aprob.')]['tipo_acta'].unique()

In [ ]:
actas[(actas['materia'] == 'Exactas Programa') & (actas['nota'] == 'No Aprob.')]['tipo_acta'].unique()

In [ ]:
actas[(actas['materia'] == 'Exactas Programa') & (actas['nota'] == 'No Aprob.')]['fecha'].unique()

In [ ]:
actas[(actas['materia'] == 'Exactas Programa') & (actas['nota'] == 'Aprob.')]['fecha'].unique()

Hay otras materias donde hacen lo mismo

In [ ]:
actas[(actas['tipo_acta'] == 'Acta de Examen') & (actas['nota'] == 'Aprob.')]['materia'].unique()

In [ ]:
actas[(actas['tipo_acta'] == 'Acta de Examen') & (actas['materia'] == 'Introducción a la Biología Molecular y Celular') & (actas['nota'] == 'Aprob.')]

In [ ]:
actas[(actas['tipo_acta'] == 'Acta de Examen') & (actas['materia'] == 'Sensores Remotos y su Aplicación en Geología') & (actas['nota'] == 'Aprob.')]

In [ ]:
actas[(actas['tipo_acta'] == 'Acta de Examen') & (actas['nota'] == 'Sobresalie')]

Si vemos las actas de TP

In [ ]:
actas[actas['tipo_acta'] == 'Acta de Regulares/Promociones']['nota'].value_counts()

Vemos unos pocos casos donde se asignaron valores numericos

In [ ]:
actas[(actas['tipo_acta'] == 'Acta de Regulares/Promociones') & ((actas['nota'] == '10') | (actas['nota'] == '9'))]

# Cantidad de materias y carreras distintas y cuales son

In [ ]:
# imprimo actas['materia'].value_counts() como markdown
print(tabulate.tabulate(actas['materia'].value_counts().reset_index(), headers=['Materia', 'Cantidad'], tablefmt='markdown'))

In [ ]:
actas['carrera'].unique()

# Analizo las fechas

Hay actas que son antes del 2020, eso por qué podría ser?

In [ ]:
actas['fecha'].min()

In [ ]:
actas[actas['fecha'] < '2020-01-01']

# Valores posibles de carrera principal

In [ ]:
personas['carrera_principal'].unique()

In [ ]:
np.isin(actas['carrera'].unique(), personas['carrera_principal'].unique()).sum()

In [ ]:
actas['carrera'][~actas['carrera'].isin(personas['carrera_principal'].unique())].value_counts()

# Los posibles niveles de estudio coinciden

In [ ]:
np.isin(personas['nivel_estudio_madre'].unique(), personas['nivel_estudio_padre'].unique()).sum()

In [ ]:
personas['nivel_estudio_madre'].nunique()

In [ ]:
personas['nivel_estudio_padre'].nunique()

# Tengo casos de riesgo de abandono registrados?

In [ ]:
actas

agrego columna del año y mes en que rindió.

In [ ]:
actas['fecha'] = pd.to_datetime(actas['fecha'], format='%Y-%m-%d')

In [ ]:
# Supongamos que tu DataFrame se llama df y la columna de fechas se llama 'fecha'
actas['año'] = actas['fecha'].dt.year
actas['mes'] = actas['fecha'].dt.month

In [ ]:
actas['semestre'] = np.where(actas['mes'] <= 6, 1, 2)

In [ ]:
actas['semestre'].value_counts()

In [ ]:
# Crear columna de periodo tipo entero para fácil comparación: 2020-1 -> 20201, 2020-2 -> 20202, etc.
actas['periodo_int'] = actas['año'] * 10 + actas['semestre']

# Función para contar si falta algún semestre
def tiene_periodos_faltantes(grupo):
    periodos = sorted(grupo['periodo_int'].unique())
    inicio = periodos[0]
    fin = periodos[-1]
    # Crear lista esperada de periodos semestrales (ej: 20201, 20202, 20211, ...)
    esperados = []
    año = inicio // 10
    semestre = inicio % 10
    while año * 10 + semestre <= fin:
        esperados.append(año * 10 + semestre)
        # avanzar al siguiente semestre
        if semestre == 1:
            semestre = 2
        else:
            semestre = 1
            año += 1
    return len(set(esperados) - set(periodos)) > 0

# Aplicar por persona
faltantes = actas.groupby('dni').apply(tiene_periodos_faltantes)

# Filtrar personas que tienen faltantes
personas_con_faltantes = faltantes[faltantes].index

# Ver cuántas son
print(f"Cantidad de personas con al menos un semestre faltante: {len(personas_con_faltantes)}")


Si hago esto mismo con datos de actas de 2020 y 2021

In [ ]:
# Función para contar si falta algún semestre
def tiene_periodos_faltantes(grupo):
    periodos = sorted(grupo['periodo_int'].unique())
    inicio = periodos[0]
    fin = periodos[-1]
    # Crear lista esperada de periodos semestrales (ej: 20201, 20202, 20211, ...)
    esperados = []
    año = inicio // 10
    semestre = inicio % 10
    while año * 10 + semestre <= fin:
        esperados.append(año * 10 + semestre)
        # avanzar al siguiente semestre
        if semestre == 1:
            semestre = 2
        else:
            semestre = 1
            año += 1
    return len(set(esperados) - set(periodos)) > 0

#me quedo con actas de 2020 y 2021
actas_2020_2021 = actas[(actas['año'] == 2020) | (actas['año'] == 2021)]

# Aplicar por persona
faltantes_2020_2021 = actas_2020_2021.groupby('dni').apply(tiene_periodos_faltantes)

# Filtrar personas que tienen faltantes
personas_con_faltantes_2020_2021 = faltantes_2020_2021[faltantes_2020_2021].index

# Ver cuántas son
print(f"Cantidad de personas con al menos un semestre faltante: {len(personas_con_faltantes_2020_2021)}")


In [ ]:
actas_2020_2021['dni'].nunique()

In [ ]:
361*100/2537

Con todo esto, tampoco estamos considerando el cbc, en principio vamos a trabajar solamente con los datos que tienen el historial del cbc

In [ ]:
en_cbc = cbc['Dni'].unique()

In [ ]:
actas_con_cbc = actas[actas['dni'].isin(en_cbc)]
personas_con_cbc = personas[personas['dni'].isin(en_cbc)]

In [ ]:
actas.shape

In [ ]:
actas_con_cbc['dni'].nunique()

In [ ]:
personas_con_cbc_y_actas = personas[personas['dni'].isin(actas_con_cbc['dni'].unique())]

In [ ]:
personas_con_cbc.shape

In [ ]:
personas_con_cbc_y_actas.shape

In [ ]:
# Función para contar si falta algún semestre
def tiene_periodos_faltantes(grupo):
    periodos = sorted(grupo['periodo_int'].unique())
    inicio = periodos[0]
    fin = periodos[-1]
    # Crear lista esperada de periodos semestrales (ej: 20201, 20202, 20211, ...)
    esperados = []
    año = inicio // 10
    semestre = inicio % 10
    while año * 10 + semestre <= fin:
        esperados.append(año * 10 + semestre)
        # avanzar al siguiente semestre
        if semestre == 1:
            semestre = 2
        else:
            semestre = 1
            año += 1
    return len(set(esperados) - set(periodos)) > 0

#me quedo con actas de 2020 y 2021
actas_2020_2021 = actas_con_cbc[(actas_con_cbc['año'] == 2020) | (actas_con_cbc['año'] == 2021) | (actas_con_cbc['año'] == 2022)]

# Aplicar por persona
faltantes_2020_2021 = actas_2020_2021.groupby('dni').apply(tiene_periodos_faltantes)

# Filtrar personas que tienen faltantes
personas_con_faltantes_2020_2021 = faltantes_2020_2021[faltantes_2020_2021].index

# Ver cuántas son
print(f"Cantidad de personas con al menos un semestre faltante: {len(personas_con_faltantes_2020_2021)}")

In [ ]:
actas_2020_2021.shape

In [ ]:
actas_con_cbc['nota'].value_counts(dropna=False)

In [ ]:
actas_con_cbc[actas_con_cbc['nota'].isna()]['tipo_acta'].unique()

In [ ]:
# Función para contar si falta algún semestre
def tiene_periodos_faltantes(grupo):
    periodos = sorted(grupo['periodo_int'].unique())
    inicio = periodos[0]
    fin = periodos[-1]
    # Crear lista esperada de periodos semestrales (ej: 20201, 20202, 20211, ...)
    esperados = []
    año = inicio // 10
    semestre = inicio % 10
    while año * 10 + semestre <= fin:
        esperados.append(año * 10 + semestre)
        # avanzar al siguiente semestre
        if semestre == 1:
            semestre = 2
        else:
            semestre = 1
            año += 1
    return len(set(esperados) - set(periodos)) > 0

#me quedo con actas de 2020 y 2021
actas_2020_2021 = actas_con_cbc[(actas_con_cbc['año'] == 2020) | (actas_con_cbc['año'] == 2021) | (actas_con_cbc['año'] == 2022)]

# Aplicar por persona
faltantes_2020_2021 = actas_2020_2021.groupby('dni').apply(tiene_periodos_faltantes)

# Filtrar personas que tienen faltantes
personas_con_faltantes_2020_2021 = faltantes_2020_2021[faltantes_2020_2021].index

# Ver cuántas son
print(f"Cantidad de personas con al menos un semestre faltante: {len(personas_con_faltantes_2020_2021)}")

In [ ]:
# Agrupamos por DNI y Materia
grupo = actas_con_cbc.groupby(['dni', 'materia'])

# Creamos una función que nos diga si tiene ambas actas y en años distintos
def tiene_ambas_actas_distinto_anio(df_grupo):
    actas = df_grupo['tipo_acta'].unique()
    if 'Acta de Regulares/Promociones' in actas and 'Acta de Examen' in actas:
        anios_regulares = df_grupo[df_grupo['tipo_acta'] == 'Acta de Regulares/Promociones']['año'].unique()
        anios_examen = df_grupo[df_grupo['tipo_acta'] == 'Acta de Examen']['año'].unique()
        return any(a != b for a in anios_regulares for b in anios_examen)
    return False

# Aplicamos la función al grupo
resultado = grupo.filter(tiene_ambas_actas_distinto_anio)

# Contamos cuántas personas únicas tienen esta situación
cantidad_personas = resultado[['dni', 'materia']].drop_duplicates()['dni'].nunique()

print(f'Cantidad de personas con ambas actas en años distintos: {cantidad_personas}')


In [ ]:
# Suponiendo que tenés las columnas: 'DNI', 'Materia', 'Acta', 'Año', 'Semestre'
df = actas_con_cbc.copy()
# Creamos una columna para ordenar los periodos (ej: 2023-S1 → 20231, 2023-S2 → 20232)
df['periodo'] = df['año'].astype(str) + df['semestre'].astype(str)
df['periodo_ordenado'] = df['año'] * 2 + df['semestre'].replace({1: 0, 2: 1})  # 2023-S1 → 4046, 2023-S2 → 4047

# Nos quedamos solo con actas relevantes
df_filtrado = df[df['tipo_acta'].isin(['Acta de Regulares/Promociones', 'Acta de Examen'])]

# Agrupamos por DNI y Materia
grupo = df_filtrado.groupby(['dni', 'materia'])

# Función para chequear si hay una diferencia de un semestre entre actas
def semestre_de_diferencia(df_grupo):
    df_r = df_grupo[df_grupo['tipo_acta'] == 'Acta de Regulares/Promociones']
    df_e = df_grupo[df_grupo['tipo_acta'] == 'Acta de Examen']
    
    for pr in df_r['periodo_ordenado']:
        for pe in df_e['periodo_ordenado']:
            if abs(pe - pr) >= 1:  # diferencia de un semestre
                return True
    return False

# Aplicamos el filtro
resultado = grupo.filter(semestre_de_diferencia)

# Obtenemos personas y materias únicas
personas_con_semestre_diferencia = resultado[['dni', 'materia']].drop_duplicates()

# Cantidad
cantidad = personas_con_semestre_diferencia['dni'].nunique()

print(f'Cantidad de personas que rinden con un semestre de diferencia: {cantidad}')

### Analisis para nuevas opciones

- Dado cada alumno, calcular cuantas materias aprobó en los cuatrimestres 0....6  donde el cuatrimestre 0 es el 1er cuatrimestre que cursó.

- Luego sumarizar 7 bins donde cada bin tiene la cantidad de materias aprobadas en el cuatrimestre_i

Si surge algo interesante podría compararse esta pendiente con la pendiente de un alumno como variable target.

* Luego sumarizar para cada bin_i la cantidad de materias aprobadas por todos los alumnos en el cuatrimestre_i
Eso va a calcular una pendiente empírica (imagino que descendiente) capturando el comportamiento promedio

In [ ]:
actas['resultado'].unique()

In [ ]:
actas['resultado'].value_counts(dropna=False)

In [ ]:
actas.columns

In [ ]:
# Ordenamos y asignamos el cuatrimestre relativo (0, 1, 2, ...) por alumno
actas = actas.sort_values(['dni', 'periodo_int'])
actas['cuatrimestre_relativo'] = actas.groupby('dni')['periodo_int'].rank(method='dense').astype(int) - 1

In [ ]:
actas

Tiene completo sentido que vaya en bajada porque de 2020 a 2024, todos tienen un "primer cuatrimestre" pero no todos llegaron a tener un segundo, tercer, cuarto, etc, cuatrimestre.

In [ ]:
# Paso 2: Filtrar solo aprobados
actas_aprobados = actas[actas['resultado'] == 'Aprobado']

# Paso 3: Contar aprobadas por alumno y cuatrimestre relativo
aprobados_por_alumno = actas_aprobados.groupby(['dni', 'cuatrimestre_relativo']).size().reset_index(name='cantidad_aprobadas')

# Paso 4: Sumarizar por cuatrimestre relativo total (lo que llamás bin_i)
aprobados_totales_por_cuatrimestre = aprobados_por_alumno.groupby('cuatrimestre_relativo')['cantidad_aprobadas'].sum().reset_index()

# Resultado final
print(aprobados_totales_por_cuatrimestre)

In [ ]:
# Graficar histograma (en realidad es un gráfico de barras porque son categorías discretas)
plt.figure(figsize=(8, 5))
plt.bar(aprobados_totales_por_cuatrimestre['cuatrimestre_relativo'],
        aprobados_totales_por_cuatrimestre['cantidad_aprobadas'],
        color='skyblue')

plt.title('Total de Materias Aprobadas por Cuatrimestre Relativo')
plt.xlabel('Cuatrimestre Relativo')
plt.ylabel('Cantidad de Aprobaciones')
plt.xticks(aprobados_totales_por_cuatrimestre['cuatrimestre_relativo'])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

Quiero eso mismo pero por año de inscripción

In [ ]:
anio_inscripcion = personas[['dni', 'año_inscripcion_facultad']]

In [ ]:
actas_con_anio_inscripcion = pd.merge(actas, anio_inscripcion, left_on='dni', right_on='dni', how='left')

In [ ]:
inscriptos_2020 = actas_con_anio_inscripcion[actas_con_anio_inscripcion['año_inscripcion_facultad'] == 2020]
# Paso 2: Filtrar solo aprobados
actas_aprobados = inscriptos_2020[(inscriptos_2020['resultado'] == 'Aprobado') & (inscriptos_2020['tipo_acta'] == 'Acta de Examen')]

# Paso 3: Contar aprobadas por alumno y cuatrimestre relativo
aprobados_por_alumno = actas_aprobados.groupby(['dni', 'cuatrimestre_relativo']).size().reset_index(name='cantidad_aprobadas')

# Paso 4: Sumarizar por cuatrimestre relativo total (lo que llamás bin_i)
aprobados_totales_por_cuatrimestre = aprobados_por_alumno.groupby('cuatrimestre_relativo')['cantidad_aprobadas'].sum().reset_index()

# Resultado final
print(aprobados_totales_por_cuatrimestre)
# Graficar histograma (en realidad es un gráfico de barras porque son categorías discretas)
plt.figure(figsize=(8, 5))
plt.bar(aprobados_totales_por_cuatrimestre['cuatrimestre_relativo'],
        aprobados_totales_por_cuatrimestre['cantidad_aprobadas'],
        color='skyblue')

plt.title('Total de Materias Aprobadas por Cuatrimestre Relativo')
plt.xlabel('Cuatrimestre Relativo')
plt.ylabel('Cantidad de Aprobaciones')
plt.xticks(aprobados_totales_por_cuatrimestre['cuatrimestre_relativo'])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
frecuencia = aprobados_por_alumno.groupby(['cuatrimestre_relativo', 'cantidad_aprobadas']) \
    .size().unstack(fill_value=0)

plt.figure(figsize=(12, 6))
sns.heatmap(frecuencia.T, cmap='YlGnBu', annot=True, fmt='d', cbar_kws={'label': 'Cantidad de Alumnos'})

plt.title('Distribución de Materias Aprobadas por Cuatrimestre Relativo')
plt.xlabel('Cuatrimestre Relativo')
plt.ylabel('Materias Aprobadas')
plt.tight_layout()
plt.show()

In [ ]:
inscriptos_2021 = actas_con_anio_inscripcion[actas_con_anio_inscripcion['año_inscripcion_facultad'] == 2021]
# Paso 2: Filtrar solo aprobados
actas_aprobados = inscriptos_2021[(inscriptos_2021['resultado'] == 'Aprobado') & (inscriptos_2021['tipo_acta'] == 'Acta de Examen')]

# Paso 3: Contar aprobadas por alumno y cuatrimestre relativo
aprobados_por_alumno = actas_aprobados.groupby(['dni', 'cuatrimestre_relativo']).size().reset_index(name='cantidad_aprobadas')

# Paso 4: Sumarizar por cuatrimestre relativo total (lo que llamás bin_i)
aprobados_totales_por_cuatrimestre = aprobados_por_alumno.groupby('cuatrimestre_relativo')['cantidad_aprobadas'].sum().reset_index()

# Resultado final
print(aprobados_totales_por_cuatrimestre)
# Graficar histograma (en realidad es un gráfico de barras porque son categorías discretas)
plt.figure(figsize=(8, 5))
plt.bar(aprobados_totales_por_cuatrimestre['cuatrimestre_relativo'],
        aprobados_totales_por_cuatrimestre['cantidad_aprobadas'],
        color='skyblue')

plt.title('Total de Materias Aprobadas por Cuatrimestre Relativo')
plt.xlabel('Cuatrimestre Relativo')
plt.ylabel('Cantidad de Aprobaciones')
plt.xticks(aprobados_totales_por_cuatrimestre['cuatrimestre_relativo'])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()

In [ ]:
frecuencia = aprobados_por_alumno.groupby(['cuatrimestre_relativo', 'cantidad_aprobadas']) \
    .size().unstack(fill_value=0)

plt.figure(figsize=(12, 6))
sns.heatmap(frecuencia.T, cmap='YlGnBu', annot=True, fmt='d', cbar_kws={'label': 'Cantidad de Alumnos'})

plt.title('Distribución de Materias Aprobadas por Cuatrimestre Relativo')
plt.xlabel('Cuatrimestre Relativo')
plt.ylabel('Materias Aprobadas')
plt.tight_layout()
plt.show()